In [4]:
import pandas as pd
import dash
from dash import html  # Updated import
from dash import dcc   # Updated import
from dash.dependencies import Input, Output
import plotly.express as px

In [10]:
# Read the SpaceX data into a pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

# Get the max and min values for payload mass for the range slider
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
# Create a dash application
app = dash.Dash(__name__)

# Create the app layout
app.layout = html.Div(children=[
    # Header
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    
    html.Br(),

    # Dropdown for selecting the launch site
    html.Div([
        dcc.Dropdown(
            id='site-dropdown',
            options=[
                {'label': 'All Launch Sites', 'value': 'ALL'},
                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
            ],
            value='ALL',  # Default value
            placeholder="Select a Launch Site",
            style={'width': '80%', 'padding': '3px', 'font-size': '20px'}
        ),
    ], style={'width': '48%', 'display': 'inline-block'}),
    
    html.Br(),

    # TASK 2: Add a pie chart for success and failure count based on selected site
    html.Div(dcc.Graph(id='success-pie-chart')),

    html.Br(),

    # TASK 3: Add a slider to filter payload range
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(
        id='payload-slider',
        min=min_payload,
        max=max_payload,
        step=1000,
        marks={i: f'{i}' for i in range(int(min_payload), int(max_payload), 5000)},
        value=[min_payload, max_payload]
    ),
    
    html.Br(),

    # TASK 4: Add scatter chart to show correlation between payload and launch success
    html.Div(dcc.Graph(id='success-payload-scatter-chart'))
])


# TASK 2: Add callback to update the success-pie-chart based on selected site
@app.callback(
    Output('success-pie-chart', 'figure'),
    [Input('site-dropdown', 'value')]
)
def update_pie_chart(selected_site):
     if selected_site == 'ALL':
        fig = px.pie(spacex_df, names='class', title='Launch Success vs Failures for All Sites',
                     labels={'class': 'Launch Outcome (0=Failure, 1=Success)', 'Payload Mass (kg)': 'Payload Mass (kg)'})
     else:
        site_data = spacex_df[spacex_df['Launch Site'] == selected_site]
        fig = px.pie(site_data, names='class', title=f'Launch Success vs Failures for {selected_site}',
                     labels={'class': 'Launch Outcome (0=Failure, 1=Success)'})
     return fig

# TASK 4: Add callback to update the success-payload scatter chart based on site and payload range
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
    Input('payload-slider', 'value')]
)
def update_scatter_chart(selected_site, payload_range):
    # Filter data based on payload range
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'] >= payload_range[0]]
    filtered_df = filtered_df[filtered_df['Payload Mass (kg)'] <= payload_range[1]]

    # If a specific site is selected, filter data for that site
    if selected_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]

    fig = px.scatter(filtered_df,
                     x='Payload Mass (kg)',
                     y='class',
                     color='Launch Site',
                     title=f"Launch Success vs Payload Mass for {selected_site if selected_site != 'ALL' else 'All Sites'}",
                     labels={'class': 'Launch Outcome (0=Failure, 1=Success)', 'Payload Mass (kg)': 'Payload Mass (kg)'},
                     category_orders={'Launch Site': ['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40']})
    fig.update_layout(autosize=True)
    return fig 


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
